In [30]:
import random
import sys
import os
import numpy as np
import matplotlib.pyplot as plt
import glob
import pickle
from tqdm import tqdm
import nibabel as nib

%matplotlib inline

In [31]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch import optim
from torchvision import transforms

from torch.utils.data import Dataset, TensorDataset, random_split, SubsetRandomSampler, ConcatDataset
from sklearn.model_selection import KFold

from sklearn.metrics import mean_absolute_error as mae

In [32]:
# Use the GPU if you have one
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [33]:
#single scale
class fc_layer(nn.Module):
    def __init__(self, in_layer, out_layer):
        super(fc_layer, self).__init__()
        self.fc = nn.Linear(in_num, out_layer)
        #if in_features=5 and out_features=10 and the input tensor x 
        #has dimensions 2-3-5, then the output tensor will have dimensions 2-3-10???

    def forward(self, x):
        return self.fc(x)

class avg_pool(nn.Module):
    def __init__(self):
        super(avg_pool, self).__init__()
        self.avgp = nn.AvgPool3d(2)
        
    def forward(self, x):
        return self.avgp(x)

class relu_act(nn.Module):
    def __init__(self):
        super(relu_act, self).__init__()
        self.relu = nn.ReLU()
    
    def forward(self, x):
        return self.relu(x)

class softmax_layer(nn.Module):
    def __init__(self):
        super(softmax_layer, self).__init__()
        self.softmax = nn.Softmax()
    
    def forward(self, x):
        return self.softmax(x)

# Age prediction

In [34]:
class age_pred(nn.Module):
    def __init__(self, in_layer, out_layer):
        super(age_pred, self).__init__()
        self.in_layer = in_layer
        self.out_layer = out_layer
        self.layer1 = fc_layer(in_layer, out_layer)
        self.avg = avg_pool()
        self.relu = relu_act()
    
    def forward(self, x):
        x0 = layer1(x)
        x1 = avg(x0)
        x2 = relu_act(x1)
        return x2

In [35]:
#loss for regression
def get_mae(model, pred, y_test):
    mae_out = mae(y_pred=pred, y_true=y_test)
    return mae_out

In [36]:
class ADNI_Dataset_regression(Dataset):
    def __init__(self, root_dir, data_file):
        """
        Args:
            root_dir (string): Directory of all the images.
            data_file (string): File name of the train/test split file.
        """
        self.root_dir = root_dir
        self.data_file = data_file
    
    def __len__(self):
        return sum(1 for line in open(self.data_file))
    
    def __getitem__(self, idx):
        df = open(self.data_file)
        lines = df.readlines()
        lst = lines[idx].split()
        img_name = lst[0]
        img_label = lst[4]  #age
        print(img_label)
        image_path = os.path.join(self.root_dir, img_name)
        image = nib.load(image_path)
        a = np.array(image.dataobj) #convert to np array
        
        sample = {'image': a, 'label': img_label}
        
        return sample

In [37]:
NUM_EPOCH = 100
BATCH_SIZE = 10
LR = 0.0001
SAVE_PATH_AP = r'C:\Users\pbhav\Desktop\NYU\ivp\project\model\AP' #path to save model
SAVE_PATH_BC = r'C:\Users\pbhav\Desktop\NYU\ivp\project\model\BC'

In [38]:
def train_epoch(net, data_loader, optimizer, criterion, epoch):
    net.train()
    loss_stat = []
    for i, img_label in enumerate(data_loader):
        img, label = img_label
        img = img.to(device=device, dtype=torch.float32)
        
        pred = net(img)
        loss = criterion(label, pred)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        loss_stat += [loss.item()]/len(data_loader) * 100
    
    print ("Epoch {}: [{}/{}] Loss: {:.3f}".format(epoch, len(data_loader), len(data_loader),np.mean(loss_stat))) 
    
    return np.mean(loss_stat)

In [39]:
def valid_epoch(net, data_loader, criterion, epoch):
    net.eval()

    val_loss_stat = []
    for i, img_label in enumerate(data_loader):
        img, label = img_label
        img = img.to(device=device, dtype=torch.float32)
        
        with torch.no_grad():
            pred = net(img)
            val_loss = criterion(label, pred)
      
        val_loss_stat += [val_loss.item()]/len(data_loader) * 100
        
    print ("Val Loss: {:.3f} ".format(np.mean(val_loss_stat)))
    
    return np.mean(val_loss_stat)

In [40]:
regression_data = ADNI_Dataset_regression(r"C:/Users/pbhav/Desktop/NYU/ivp/project/ADNI/", "C:/Users/pbhav/Downloads/ADNI1_Annual_2_Yr_3T_4_23_2022.csv")

In [ ]:
in_size = ... #size after the transformer
net = age_pred(in_size, 1)
net.to(device)  # run net.to(device) if using GPU
print(net)`

n_params = sum(p.numel() for p in net.parameters() if p.requires_grad)
print('Number of parameters in network: ', n_params)

In [ ]:
#kfold validation for k = 5
k=5
splits=KFold(n_splits=k,shuffle=True,random_state=42)

# criterion = get_mae()

In [ ]:
def k_fold(model, dataset, NUM_EPOCH, LR, BATCH_SIZE, SAVE_PATH, criterion):
    foldperf={}
    for fold, (train_idx, val_idx) in enumerate(splits.split(np.arange(len(dataset)))):

        print('Fold {}'.format(fold + 1))

        train_sampler = SubsetRandomSampler(train_idx)
        test_sampler = SubsetRandomSampler(val_idx)
        train_loader = DataLoader(dataset, batch_size=BATCH_SIZE, sampler=train_sampler)
        test_loader = DataLoader(dataset, batch_size=BATCH_SIZE, sampler=test_sampler)

        device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

        model = model()
        model.to(device)
        optimizer = optim.Adam(model.parameters(), lr=LR)

        history = {'train_loss': [], 'test_loss': [],'train_acc':[],'test_acc':[]}

        for epoch in range(NUM_EPOCH):
            train_loss, train_correct=train_epoch(model, train_loader, optimizer, criterion, epoch)
            test_loss, test_correct=valid_epoch(model, test_loader, criterion, epoch)

            train_loss = train_loss / len(train_loader.sampler)
            train_acc = train_correct / len(train_loader.sampler) * 100
            test_loss = test_loss / len(test_loader.sampler)
            test_acc = test_correct / len(test_loader.sampler) * 100

            print("Epoch:{}/{} AVG Training Loss:{:.3f} AVG Test Loss:{:.3f} AVG Training Acc {:.2f} % AVG Test Acc {:.2f} %".format(epoch + 1,
                                                                                                                 num_epochs,
                                                                                                                 train_loss,
                                                                                                                 test_loss,
                                                                                                                 train_acc,
                                                                                                                 test_acc))
            history['train_loss'].append(train_loss)
            history['test_loss'].append(test_loss)
            history['train_acc'].append(train_acc)
            history['test_acc'].append(test_acc)

    #         Save the model after each epoch
            if os.path.isdir(SAVE_PATH):
                torch.save(net.state_dict(),SAVE_PATH + 'agepredepoch{}.pth'.format(epoch + 1))
            else:
                os.makedirs(model_save_path, exist_ok=True)
                torch.save(net.state_dict(),SAVE_PATH + 'agepredepoch{}.pth'.format(epoch + 1))
            print('Checkpoint {} saved to {}'.format(epoch + 1, SAVE_PATH + 'agepredepoch{}.pth'.format(epoch + 1)))   
        foldperf['fold{}'.format(fold+1)] = history  

    torch.save(model,'k_cross.pt')
    return foldperf

In [ ]:
foldperf1 = k_fold(age_pred, regression_data, NUM_EPOCH, LR, BATCH_SIZE, SAVE_PATH_AP, criterion=get_mae())

In [ ]:
testl_f, tl_f, testa_f, ta_f = [], [], [], []

for f in range(1,k+1):
    tl_f.append(np.mean(foldperf1['fold{}'.format(f)]['train_loss']))
    testl_f.append(np.mean(foldperf1['fold{}'.format(f)]['test_loss']))
    ta_f.append(np.mean(foldperf1['fold{}'.format(f)]['train_acc']))
    testa_f.append(np.mean(foldperf1['fold{}'.format(f)]['test_acc']))

print('Performance of {} fold cross validation'.format(k))
print("Average Training Loss: {:.3f} \t Average Test Loss: {:.3f} \t Average Training Acc: {:.2f} \t Average Test Acc: {:.2f}".format(np.mean(tl_f), np.mean(testl_f), np.mean(ta_f), np.mean(testa_f)))     


# Brain disease classification

In [ ]:
#data prep
class ADNI_Dataset_classification(Dataset):
    def __init__(self, root_dir, data_file):
        """
        Args:
            root_dir (string): Directory of all the images.
            data_file (string): File name of the train/test split file.
        """
        self.root_dir = root_dir
        self.data_file = data_file
    
    def __len__(self):
        return sum(1 for line in open(self.data_file))
    
    def __getitem__(self, idx):
        df = open(self.data_file)
        lines = df.readlines()
        lst = lines[idx].split()
        img_name = lst[1]
        img_label = lst[2]
        image_path = os.path.join(self.root_dir, img_name)
        image = nib.load(image_path)
        a = np.array(image.dataobj) #convert to np array
        
        if img_label == 'CN': #Cognitive Normal
            label = 0
        elif img_label == 'AD': #Alzheimer's 
            label = 1
        elif img_label == 'MCI': #Mild Cognitive Impairement
            label = 2

        sample = {'image': a, 'label': label}
        
        return sample

In [ ]:
#loss function for classification
def a_value(probabilities, zero_label=0, one_label=1):
    """
    Approximates the AUC by the method described in Hand and Till 2001,
    equation 3.
    NB: The class labels should be in the set [0,n-1] where n = # of classes.
    The class probability should be at the index of its label in the
    probability list.
    I.e. With 3 classes the labels should be 0, 1, 2. The class probability
    for class '1' will be found in index 1 in the class probability list
    wrapped inside the zipped list with the labels.
    Args:
        probabilities (list): A zipped list of the labels and the
            class probabilities in the form (m = # data instances):
             [(label1, [p(x1c1), p(x1c2), ... p(x1cn)]),
              (label2, [p(x2c1), p(x2c2), ... p(x2cn)])
                             ...
              (labelm, [p(xmc1), p(xmc2), ... (pxmcn)])
             ]
        zero_label (optional, int): The label to use as the class '0'.
            Must be an integer, see above for details.
        one_label (optional, int): The label to use as the class '1'.
            Must be an integer, see above for details.
    Returns:
        The A-value as a floating point.
    """
    # Obtain a list of the probabilities for the specified zero label class
    expanded_points = []
    for instance in probabilities:
        if instance[0] == zero_label or instance[0] == one_label:
            expanded_points.append((instance[0], instance[1][zero_label]))
    sorted_ranks = sorted(expanded_points, key=lambda x: x[1])

    n0, n1, sum_ranks = 0, 0, 0
    # Iterate through ranks and increment counters for overall count and ranks of class 0
    for index, point in enumerate(sorted_ranks):
        if point[0] == zero_label:
            n0 += 1
            sum_ranks += index + 1  # Add 1 as ranks are one-based
        elif point[0] == one_label:
            n1 += 1
        else:
            pass  # Not interested in this class

    return (sum_ranks - (n0*(n0+1)/2.0)) / float(n0 * n1)  # Eqn 3

def get_mAUC(data, num_classes):
    """
    Calculates the MAUC over a set of multi-class probabilities and
    their labels. This is equation 7 in Hand and Till's 2001 paper.
    NB: The class labels should be in the set [0,n-1] where n = # of classes.
    The class probability should be at the index of its label in the
    probability list.
    I.e. With 3 classes the labels should be 0, 1, 2. The class probability
    for class '1' will be found in index 1 in the class probability list
    wrapped inside the zipped list with the labels.
    Args:
        data (list): A zipped list (NOT A GENERATOR) of the labels and the
            class probabilities in the form (m = # data instances):
             [(label1, [p(x1c1), p(x1c2), ... p(x1cn)]),
              (label2, [p(x2c1), p(x2c2), ... p(x2cn)])
                             ...
              (labelm, [p(xmc1), p(xmc2), ... (pxmcn)])
             ]
        num_classes (int): The number of classes in the dataset.
    Returns:
        The MAUC as a floating point value.
    """
    # Find all pairwise comparisons of labels
    class_pairs = [x for x in itertools.combinations(xrange(num_classes), 2)]

    # Have to take average of A value with both classes acting as label 0 as this
    # gives different outputs for more than 2 classes
    sum_avals = 0
    for pairing in class_pairs:
        sum_avals += (a_value(data, zero_label=pairing[0], one_label=pairing[1]) +
                      a_value(data, zero_label=pairing[1], one_label=pairing[0])) / 2.0

    return sum_avals * (2 / float(num_classes * (num_classes-1)))  # Eqn 7

#CITE THE PAPER AND GITHUB https://gist.github.com/stulacy/672114792371dc13b247

In [ ]:
class BD_classify(nn.Module):
    def __init__(self, in_layer, out_layer):
        super(BD_classify, self).__init__()
        self.in_layer = in_layer
        self.out_layer = out_layer
        self.layer1 = fc_layer(in_layer, out_layer)
        self.avg = avg_pool()
        self.softmax = softmax_layer()
    
    def forward(self, x):
        x0 = layer1(x)
        x1 = avg(x0)
        x2 = softmax_layer(x1)
        return x2

In [ ]:
in_size = ...#size after the transformer
net1 = BD_classify(in_size, 3)
net1.to(device)  # run net.to(device) if using GPU
print(net1)

n_params1 = sum(p.numel() for p in net1.parameters() if p.requires_grad)
print('Number of parameters in network: ', n_params1)

In [ ]:
foldperf2 = k_fold(BD_classify, ADNI_Dataset_classification, NUM_EPOCH, LR, BATCH_SIZE, SAVE_PATH_BC, criterion=get_mAUC())

In [ ]:
testl_f1, tl_f1, testa_f1, ta_f1 = [], [], [], []

for f in range(1,k+1):
    tl_f1.append(np.mean(foldperf2['fold{}'.format(f)]['train_loss']))
    testl_f1.append(np.mean(foldperf2['fold{}'.format(f)]['test_loss']))
    ta_f1.append(np.mean(foldperf2['fold{}'.format(f)]['train_acc']))
    testa_f1.append(np.mean(foldperf2['fold{}'.format(f)]['test_acc']))

print('Performance of {} fold cross validation'.format(k))
print("Average Training Loss: {:.3f} \t Average Test Loss: {:.3f} \t Average Training Acc: {:.2f} \t Average Test Acc: {:.2f}".format(np.mean(tl_f1), np.mean(testl_f1), np.mean(ta_f1), np.mean(testa_f1)))     
